In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 6.2 Measurement of current-voltage characteristic

In [ ]:
U, I = np.genfromtxt("data/IV.txt",unpack=True)

In [ ]:
plt.plot(U,I)

# 6.3 Pedestals and noise

## a) Plot an overview of the pedestals and noise for each strip

In [ ]:
# Import matrix of events and channels where axis 0 represents the channels, axis 1 the events
ADC_ik = np.genfromtxt("data/Pedestal.txt",delimiter=";")

# Calculate pedestals (should be ~500)
P = ADC_ik.sum(axis=1)/ADC_ik.shape[1]
plt.plot(P)

In [ ]:
# Calculate Common Mode Shift (should be gaussian distributed around 0)
D = (ADC_ik.T - P).T.sum(axis=0)/len(P)

plt.hist(D)

In [ ]:
# Calculate Noise per channel from ADC counts, Pedestals and CMS
Noise = np.sqrt( (( (ADC_ik.T - P).T - D)**2).sum(axis=1) / (len(D)-1))
plt.plot(Noise)

## b) Graphically represent the values of the Common Mode in a meaningful way

In [ ]:
plt.hist(D)

# 6.4 Calibration measurements

## a) Plot the measured values of the _Calibration Runs_ and their mean values

In [ ]:
# Import all pulse charges/determined energies
# (Also apply fix to convert commas as decimal delimiter to dots, what idiot wrote the program that saves the data like that???)
def conv(x):
    return x.replace(',', '.').encode()

pulse_charge, counts_C20 = np.genfromtxt((conv(x) for x in open("data/Calib/Calib_C20.txt")),unpack=True,skip_header=2)
counts_C30 = np.genfromtxt((conv(x) for x in open("data/Calib/Calib_C30.txt")),usecols=1,skip_header=2)
counts_C40 = np.genfromtxt((conv(x) for x in open("data/Calib/Calib_C40.txt")),usecols=1,skip_header=2)
counts_C50 = np.genfromtxt((conv(x) for x in open("data/Calib/Calib_C50.txt")),usecols=1,skip_header=2)
counts_C60 = np.genfromtxt((conv(x) for x in open("data/Calib/Calib_C60.txt")),usecols=1,skip_header=2)
counts_0VC20 = np.genfromtxt((conv(x) for x in open("data/Calib/Calib_C20_0V.txt")),usecols=1,skip_header=2)

In [ ]:
counts_mean = (counts_C20+counts_C30+counts_C40+counts_C50+counts_C60)/5

plt.plot(pulse_charge, counts_C20,"x")
plt.plot(pulse_charge, counts_C30,".")
plt.plot(pulse_charge, counts_C40,"o")
plt.plot(pulse_charge, counts_C50,"v")
plt.plot(pulse_charge, counts_C60,"+")
plt.plot(pulse_charge, counts_0VC20,"x")
plt.plot(pulse_charge, counts_mean,"b-")
plt.plot()

## b) Determine the dependene of the injected charge on the measured ADC values with a 4th degree polynomial

In [ ]:
poly_par, poly_cov = np.polyfit(pulse_charge,counts_mean,4,cov=True)


poly = np.poly1d(poly_par)

x = np.linspace(0,260000,10000)

plt.plot(pulse_charge,counts_mean,".")
plt.plot(x,poly(x))

## c) Compare the curve at 0 V and above the depletion voltage

In [ ]:
poly_par_0V, poly_cov_0V = np.polyfit(pulse_charge,counts_0VC20,4,cov=True)
poly_0V = np.poly1d(poly_par_0V)

plt.plot(pulse_charge,counts_mean,".")
plt.plot(x,poly(x))
plt.plot(pulse_charge,counts_0VC20,".")
plt.plot(x,poly_0V(x))